In [0]:
\cd /home/kx/mb/
\l ../kafka/kfk.q

kfk_cfg:(!) . flip(
    (`metadata.broker.list;`kafka:9092);
    (`group.id;`0);
    (`fetch.wait.max.ms;`10);
    (`statistics.interval.ms;`10000)
    );
client:.kfk.Consumer[kfk_cfg];

// Topics to subscribe to
topic1:`test1; topic2:`test2;

// Define datasets and topic callbacks for individual
// topic subscriptions `topic1 and `topic2
data1:();
topcb1:{[msg]
  msg[`data]:"c"$msg[`data];
  msg[`rcvtime]:.z.p;
  data1,::enlist msg;}

data2:();
topcb2:{[msg]
  msg[`data]:"c"$msg[`data];
  msg[`rcvtime]:.z.t;
  data2,::enlist msg;
  show msg;
  };

// Subscribe to topic1 and topic2 with different callbacks from a single client
.kfk.Subscribe[client;topic1;enlist .kfk.PARTITION_UA;topcb1]
.kfk.Subscribe[client;topic2;enlist .kfk.PARTITION_UA;topcb2]

client_meta:.kfk.Metadata[client];
show client_meta`topics;

[0;31m/home/kx/.conda/envs/kx/q/l64/libkfk.so: undefined symbol: rd_kafka_type[0m: [0;31m/home/kx/.conda/envs/kx/q/l64/libkfk.so: undefined symbol: rd_kafka_type[0m

In [1]:
\cd /home/kx/kafka

In [1]:
\l kfk.q

[0;31m/home/kx/.conda/envs/kx/q/l64/libkfk.so: undefined symbol: rd_kafka_type[0m: [0;31m/home/kx/.conda/envs/kx/q/l64/libkfk.so: undefined symbol: rd_kafka_type[0m

In [4]:
\pwd

"/home/kx/mb/kafka/kafka"


In [2]:
\ls kafka

"CONTRIBUTING.md"
"LICENSE"
"Makefile"
"README.md"
"build"
"examples"
"kfk.c"
"kfk.q"
"socketpair.c"


In [1]:
\p 5001

In [2]:
// create pubsub handle
.tplant.pshandle:hopen`::5002;

In [ ]:
events:([eId: `long$()]
  intime:`datetime$();
  time:`datetime$(); 
  source:`symbol$();
  chan:`symbol$();
  entity:`symbol$();
  datum:();
  );

In [ ]:
getbatch :{[batchSize]
    batchSize:min[maxBatchSize;batchSize];
    :select[batchSize;<intime] from events;
    };

In [ ]:
ackbatch :{[eventIds]
    //TODO sanitization/serialization
    :delete from `events where eId in eventIds;
    };

In [ ]:
addevents :{[events]
    // TODO sanitization/serialization
    events,:events;
    .tplant.pshandle("emit";events);
    };

In [ ]:
//read from kafka vs server